In [1]:
import os
import re
from bs4 import BeautifulSoup
from pathlib import Path
from collections import Counter

def extract_person_info(text):
    """이름/직책/부서 형식에서 정보 추출"""
    text = re.sub(r'\d+', '', text).strip()
    parts = text.split('/')
    if len(parts) >= 3:
        return {
            'name': parts[0].strip(),
            'positionName': parts[1].strip(),
            'deptName': parts[2].strip()
        }
    return None

def check_duplicate_comments(html_path):
    """HTML 파일에서 동일인물의 중복 의견 확인"""
    with open(html_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
    
    # user_spans에서 이름 추출
    names = []
    user_spans = soup.find_all('span', class_='user')
    
    for user_span in user_spans:
        name_elem = user_span.find('span', class_='F_12_black_b')
        if name_elem:
            info = extract_person_info(name_elem.get_text(strip=True))
            if info:
                names.append(info['name'])
    
    # 중복 체크
    name_counts = Counter(names)
    duplicates = {name: count for name, count in name_counts.items() if count > 1}
    
    return names, duplicates

def scan_all_files(base_path):
    """모든 HTML 파일 스캔"""
    approval_path = Path(base_path) / '결재'
    
    if not approval_path.exists():
        print(f"경로를 찾을 수 없습니다: {approval_path}")
        return
    
    html_files = list(approval_path.rglob('*.html'))
    print(f"총 {len(html_files)}개의 HTML 파일 스캔 중...\n")
    
    files_with_duplicates = []
    total_duplicates = 0
    
    for idx, html_file in enumerate(html_files, 1):
        try:
            names, duplicates = check_duplicate_comments(html_file)
            
            if duplicates:
                files_with_duplicates.append({
                    'file': html_file.name,
                    'path': str(html_file),
                    'duplicates': duplicates,
                    'all_names': names
                })
                total_duplicates += 1
            
            if idx % 100 == 0:
                print(f"진행: {idx}/{len(html_files)} (중복 발견: {total_duplicates}건)")
                
        except Exception as e:
            print(f"오류 ({html_file.name}): {e}")
    
    # 결과 출력
    print(f"\n{'='*60}")
    print(f"🔍 스캔 완료: {len(html_files)}개 파일")
    print(f"⚠️  중복 발견: {total_duplicates}개 파일")
    print(f"{'='*60}\n")
    
    if files_with_duplicates:
        print("📋 중복 의견이 있는 파일 목록:\n")
        for item in files_with_duplicates[:20]:  # 처음 20개만
            print(f"📄 파일: {item['file']}")
            print(f"   경로: {item['path']}")
            for name, count in item['duplicates'].items():
                print(f"   👤 {name}: {count}번 등장")
            print(f"   전체: {item['all_names']}")
            print()
        
        if len(files_with_duplicates) > 20:
            print(f"... 외 {len(files_with_duplicates) - 20}개 파일 더 있음\n")
    else:
        print("✅ 모든 파일에서 중복 없음!")
    
    return files_with_duplicates


if __name__ == '__main__':
    base_path = r'C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html'
    results = scan_all_files(base_path)
    
    # 결과를 파일로 저장하고 싶다면
    if results:
        with open('duplicate_comments.txt', 'w', encoding='utf-8') as f:
            for item in results:
                f.write(f"파일: {item['file']}\n")
                f.write(f"경로: {item['path']}\n")
                for name, count in item['duplicates'].items():
                    f.write(f"  - {name}: {count}번\n")
                f.write(f"  전체: {item['all_names']}\n\n")
        print("📝 결과가 'duplicate_comments.txt'에 저장되었습니다.")

총 840개의 HTML 파일 스캔 중...

진행: 100/840 (중복 발견: 1건)
진행: 200/840 (중복 발견: 3건)
진행: 300/840 (중복 발견: 5건)
진행: 400/840 (중복 발견: 5건)
진행: 500/840 (중복 발견: 6건)
진행: 600/840 (중복 발견: 6건)
진행: 700/840 (중복 발견: 6건)
진행: 800/840 (중복 발견: 6건)

🔍 스캔 완료: 840개 파일
⚠️  중복 발견: 6개 파일

📋 중복 의견이 있는 파일 목록:

📄 파일: 20100220_아웃소싱 인원 계약 진행 품의-한국한의학연구원(박기영,오경진)_2002153.html
   경로: C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\결재\2010\20100220_아웃소싱 인원 계약 진행 품의-한국한의학연구원(박기영,오경진)_2002153.html
   👤 CEO: 4번 등장
   전체: ['김하응', '고화석', 'CEO', 'CEO', 'CEO', 'CEO']

📄 파일: 20100303_아웃소싱 인건비 비용 결제 품의-한국전기연구원(이동언)_2002182.html
   경로: C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\결재\2010\20100303_아웃소싱 인건비 비용 결제 품의-한국전기연구원(이동언)_2002182.html
   👤 CEO: 4번 등장
   전체: ['CEO', 'CEO', 'CEO', 'CEO']

📄 파일: 20100331_식약청 프로젝트 비용지급 품의_2002265.html
   경로: C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\결재\2010\20100331_식약청 프로젝트 비용지급 품의_2002265.html
   👤 CEO: 4번 등장
   전체: ['김동력', 'CEO', 'CEO', 'CEO', 'CEO']

📄 파일: 20100408_메디슨